# TODO

1. Can't use `dlim` directly in a JupyterLab-based linux command without full path, even though its location is in `$PATH`. 
2. Some dlim commands require confirmation (i.e., user input) so running from JupyterLab is problematic.

# Deploy model on WMLA

- Requires WMLA CLI `dlim` (patch from dev team)
- Suggestion to place in `$HOME/bin`
- Make executable `chmod a+x $HOME/bin/dlim`
- Suggestion to place on path `export PATH="$PATH:$HOME/bin"`
- Sample usage `dlim model list --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN`

# 1. Setup

In [5]:
import os
import json
import requests

In [127]:
# REST_SERVER required by dlim CLI tool
!HOST=https://wmla-console-cpd-wmla.apps.cpd.mskcc.org 
!REST_SERVER=https://wmla-console-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/

# 2. Set up WMLA deployment dependencies

## 2.1 Create working directory

In [141]:
# Deployment name
%env DEPLOY_NAME=pingpong1
%env DIR_DEPLOY_SUBMISSION=/userfs/deploy_submissions/pingpong1

!rm -rf $DIR_DEPLOY_SUBMISSION
!mkdir -p $DIR_DEPLOY_SUBMISSION

env: DEPLOY_NAME=pingpong1
env: DIR_DEPLOY_SUBMISSION=/userfs/deploy_submissions/pingpong1


## 2.2 Create model JSON file

In [143]:
model_profile = {
    "name" : os.environ['DEPLOY_NAME'],
    "tag" : "test",
    "weight_path" : "N/A",
    "runtime" : "python",
    "kernel_path" : "kernel.py",
    "schema_version" : "1"
}

# Save to working directory
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/model.json','w') as f:
    json.dump(model_profile, f)

## 2.3 Create README file

In [145]:
with open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/README.md','w') as f:
    f.write('Test.')

## 2.4 Create kernel file

In [150]:
kernel_file = """#!/usr/bin/env python

import redhareapiversion
from redhareapi import Kernel
import json

class TestKernel(Kernel):

    def on_task_invoke(self, task_context):
        try:
            Kernel.log_debug("on_task_invoke")
            # using loop to handle batch-size
            while task_context != None:
                Kernel.log_debug("task id: %s " % task_context.get_id())
                input_data = json.loads(task_context.get_input_data())
                if 'seq' in input_data:
                    Kernel.log_debug("sequence number: %s " % input_data['seq'])
                task_context.set_output_data(json.dumps(input_data))
                task_context = task_context.next()
        except Exception as e:
            Kernel.log_error(str(e))

if __name__ == '__main__':
    ppkernel = TestKernel()
    ppkernel.run()"""

print(kernel_file, file=open(f'{os.environ["DIR_DEPLOY_SUBMISSION"]}/kernel.py', 'w'))

## 2.5 Submit deployment

In [155]:
# Remove hidden files
!rm -rf $DIR_DEPLOY_SUBMISSION/.*

rm: refusing to remove '.' or '..' directory: skipping '/userfs/deploy_submissions/pingpong1/.'
rm: refusing to remove '.' or '..' directory: skipping '/userfs/deploy_submissions/pingpong1/..'


In [157]:
!$HOME/bin/dlim model deploy -p $DIR_DEPLOY_SUBMISSION --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Uploading...
</userfs/deploy_submissions/pingpong1/README.md> uploaded to server.
</userfs/deploy_submissions/pingpong1/kernel.py> uploaded to server.
</userfs/deploy_submissions/pingpong1/model.json> uploaded to server.
Registering...
Model <pingpong1> is deployed successfully


In [158]:
## ensure we see deployment
!$HOME/bin/dlim model list --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

NAME                      REST URI
deepliif-deployment-test  -
ping-pong-test-1          -
pingpong                  https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/pingpong
pingpong1                 -


In [159]:
!$HOME/bin/dlim model view $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Name:		pingpong1
Tag:		test
Model path:	/opt/wml-edi/repo/pingpong1/pingpong1-20220126-230939
Size:		0.95KB
Weight path:	N/A
Runtime:	python
Kernel path:	kernel.py
Creator:	nietor
Create time:	Wed Jan 26 23:09:39 UTC 2022
Update time:	Wed Jan 26 23:09:39 UTC 2022
REST URI:	-
Attributes:	No attribute defined
Environments:	No environment variable defined
Schema version:	1


In [160]:
!$HOME/bin/dlim model viewprofile $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Name:                       pingpong1
Replica:                    1
Create Time:                Wed Jan 26 23:09:39 2022 GMT
Last Update Time:           Wed Jan 26 23:09:39 2022 GMT
Schedule Interval:          3
Kernel Min:                 1
Kernel Max:                 100
Kernel Delay Release Time:  60
Task Execution Timeout:     60
Task Batch Size:            1
Task Parallel Size:         1
Task Pipe Size:             1
Stream Number per Group:    0
Stream Discard Slow Tasks:  true
Service Resource Group:     
Service Consumer:           
Service RG Requirement:     
Kernel Resource Group:      
Kernel Consumer:            
Kernel RG Requirement:      
Working Directory:          ${REDHARE_TOP}/work
Logs Directory:             ${REDHARE_TOP}/logs
Connection Timeout:         90
GPU:                        no
Environments:              
-                           DLIM_MK_LOG_LEVEL = INFO


## 2.6 Start deployment

In [161]:
!$HOME/bin/dlim model start $DEPLOY_NAME --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Starting model "pingpong1", run "dlim model view pingpong1 -s" to ensure startup.


In [164]:
!$HOME/bin/dlim model view $DEPLOY_NAME -s --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Name:             pingpong1
State:            Started
Serving replica:  1
Serving service ID:   fe0b9f13-f486-4686-8808-6729d4de6649
Service JobID:        edi-pingpong1-546c975c75-ppp52
GPU Mode:             no
Served clients:       0
Pending requests:     0
Requests per second:  0.00
Data per second:      0.00
Kernel started:       1


## 2.6 Obtain REST URI

In [169]:
!$HOME/bin/dlim model view $DEPLOY_NAME -a --rest-server $REST_SERVER --jwt-token $USER_ACCESS_TOKEN

Name:		pingpong1
Tag:		test
Model path:	/opt/wml-edi/repo/pingpong1/pingpong1-20220126-230939
Size:		0.95KB
Weight path:	N/A
Runtime:	python
Kernel path:	kernel.py
Creator:	nietor
Create time:	Wed Jan 26 23:09:39 UTC 2022
Update time:	Wed Jan 26 23:09:39 UTC 2022
REST URI:	https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/pingpong1
Attributes:	No attribute defined
Environments:	No environment variable defined
Schema version:	1


# 3. Score against deployment

In [179]:
headers = {'Authorization': f'Bearer {os.getenv("USER_ACCESS_TOKEN")}'}
data = {'data':'123'}
url = f'https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/{os.environ["DEPLOY_NAME"]}'

In [180]:
r = requests.post(url, headers=headers,
                  json = data, verify = False)
r.status_code

/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wmla-inference-cpd-wmla.apps.cpd.mskcc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200

In [183]:
r.json()

{'data': '123'}

# 4. Clean up / Miscellaneous

- **NOTE**: These commands require user input (i.e., confirmation) so cannot run from JupyterLab